In [22]:
!pip install pytesseract

In [67]:
import cv2
import pandas as pd
from ultralytics import YOLO
import cvzone
import numpy as np
import pytesseract
from datetime import datetime

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

model = YOLO('license_best.pt')

def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        point = [x, y]
        print(point)

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap = cv2.VideoCapture(r"C:\Users\aliye\OneDrive\Desktop\tests\work3.mp4")

my_file = open(r"C:\Users\aliye\OneDrive\Desktop\tests\plates\classes.txt", "r")
data = my_file.read()
class_list = data.split("\n") 

area = [(720, 600), (920, 600), (920, 620), (720, 620)]   # group 1 corresponds to top right, group 2 corresponds to top left, 
                                                          # group 3 corresponds to bottom left, group 4 corresponds to bottom right
count = 0
list1 = []
processed_numbers = set()

# Open file for writing car plate data
car_plate_data = []

while True:    
    ret, frame = cap.read()
    count += 1
    if count % 3 != 0:
        continue
    if not ret:
       break
   
    frame = cv2.resize(frame, (1920, 1080))
    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.cpu()
    px = pd.DataFrame(a).astype("float")
   
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        
        d = int(row[5])
        c = class_list[d]
        cx = int(x1 + x2) // 2
        cy = int(y1 + y2) // 2
        result = cv2.pointPolygonTest(np.array(area, np.int32), ((cx, cy)), False)
        if result >= 0:
           crop = frame[y1:y2, x1:x2]
           gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
           gray = cv2.bilateralFilter(gray, 10, 20, 20)

           text = pytesseract.image_to_string(gray).strip()
           text = text.replace('(', '').replace(')', '').replace(',', '')
           if text not in processed_numbers:
              processed_numbers.add(text) 
              list1.append(text)
              current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
              date, time = current_datetime.split(' ')
              car_plate_data.append([text, date, time])
              cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
              cv2.imshow('crop', crop)

      
    cv2.polylines(frame, [np.array(area, np.int32)], True, (255, 0, 0), 2)
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()    
cv2.destroyAllWindows()

# Create a DataFrame from the collected data
car_plate_df = pd.DataFrame(car_plate_data, columns=['NumberPlate', 'Date', 'Time']).assign(**{'Violation type': 'driving in opposite direction'})

# Save the DataFrame to an Excel file
car_plate_df.to_excel('C:\\Users\\aliye\\OneDrive\\Desktop\\tests\\car_plate_data.xlsx', index=False)



0: 384x640 1 license_plate, 468.5ms
Speed: 3.0ms preprocess, 468.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
[983, 558]
[911, 445]
[911, 445]

0: 384x640 1 license_plate, 82.7ms
Speed: 2.0ms preprocess, 82.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 76.1ms
Speed: 2.0ms preprocess, 76.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 77.9ms
Speed: 1.0ms preprocess, 77.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 80.6ms
Speed: 1.9ms preprocess, 80.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 80.5ms
Speed: 1.0ms preprocess, 80.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 79.6ms
Speed: 1.9ms preprocess, 79.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 77.9m